# Räkna vattendrag i Sverige

I den här uppgiften ska vi räkna hur många vattendrag det finns i en satellibild av en del av Sverige.

Programmeringsmässigt går den här uppgiften rätt långt på djupet, där vi lär oss om djupetförst-sökning (DFS). 

Genom att köra den dolda kodrutan nedan kan du se vilket område vi kommer titta på i satellitbilden:

In [0]:
#@title Kod för att rita karta
import folium
m = folium.Map(
    location=[66,17.75],
    tiles='Stamen Terrain',
    zoom_start=9
)

folium.Rectangle(
    bounds=[(65.79,17.22), (66.26,18.31)],
    fill=False
).add_to(m)


m

Zooma ut så att du ser var i Sverige bilden är tagen.

I den här uppgiften ska vi försöka räkna hur många sjöar det är i området. Utzoomat kanske det ser ut som att det bara är en eller några få sjöar. Zoomar man in så ser man dock att det dyker upp betydligt fler och de som tidigare såg ut att hänga ihop inte gör det.

Det kommer i princip vara omöjligt att räkna antalet sjöar för hand, därför ska vi ta hjälp av programmering!

Det vi kollade på ovan var en karta. För att ladda in den riktiga satellitbilden behöver vi köra koden här nedanför.

In [0]:
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/lake/lake.npz --quiet
import numpy as np
import matplotlib.pyplot as plt

bands = np.load('lake.npz')

# Visa skalor i ovankant av grafer:
plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True

Innehållet i filen vi laddade ner har lagts in i variabeln `bands`. Satellitbilden är inte en bild, utan flera bilder för olika band. Ett band betyder att man har tagit en bild men bara för ljus med vissa våglängder.

`Bands` är en dictionary som innehåller mätningar för de olika banden. Vi kan se vilka band som finns genom att skriva ut nycklarna.

In [0]:
print(list(bands.keys()))

Vi har rött, grönt blått och nära infrarött ljus att jobba med. Det nära infraröda ljuset kallar vi för ```nir``` (near infrared). Nära infrarött ljus kan vi inte se med våra mänskliga ögon men det kan vara till stor hjälp ändå. I denna uppgift kommer vi utnyttja att vatten inte reflekterar nära infrarött men reflekterar små mängder rött ljus. Detta gör att vi kommer kunna skilja vatten från växtlighet och bebyggelse.

Vi sparar varje band i vars en variabel så att det blir lättare att jobba med.

In [0]:
red  =bands["red"]
blue =bands["blue"]
green=bands["green"]
nir  =bands["nir"]


Så vad innehåller de här banden? Skriv ut något av dem för att se vad de innehåller.

In [0]:
# Skriv ut något av banden
print(...)

Varje band verkar bara vara en stor tabell med tal. Kanske verkar det konstigt att detta kan vara en bild, men det är faktiskt sant. 

Varje tal i tabellen motsvarar en pixel, alltså en liten ruta. Talet säger oss hur ljust det ska vara i just den pixeln, för ljusfrekvensen som bandet har mätt på. 

Vi kan göra en bild av till exempel det gröna bandet med koden nedan.



In [0]:
plt.figure(figsize=(10,10))
plt.imshow(green)
plt.ylim(green.shape[0], 0)
plt.xlim(0, green.shape[0])
plt.show()

Det ser inte riktigt ut som en vanlig kamerabild, men du kanske kan se att det är samma område som på karta längre upp. 

Att det inte ser ut som en vanlig bild beror på att vi bara kollat på det gröna ljuset. Sen har `matplotlib.pyplot`, eller `plt`, valt en egen färgskala när den visar bilden.

För att få en vanlig bild behöver vi använda rött, grönt och blått. För var och en av färgerna kan vi välja ett tal mellan 0 och 255 som är hur mycket den färgen ska vara med i en pixel. Genom att kombinera olika mycket av de olika färgerna så kan man skapa alla färger. Till exempel om man har 255 rött, 0 grönt och 255 blått så får man lila. 0 i allt blir svart, 255 i allt blir vitt. Såhär funkar i princip alla färgskärmar, troligen skärmen du använder just nu. 

I princip så motsvarar intervallet 0 till 255 också talen som är uppmätta för varje band. Som du kanske såg så var dock talen uppmätta för banden betydligt större (det beror lite på vilket band du skrev ut i en tidigare uppgift, men det finns i alla fall tal som är större än 1000). Därför måste vi omvandla de stora talen till mindre. Det gör koden nedan. Kör den så får du en bild. Det kan ta 10 - 20 sekunder.

In [0]:
from PIL import Image
rgb = np.zeros([1000, 1000, 3], dtype=np.uint8)

divide = 8 # Testa att ändra! Bör vara mellan 3 och 30 ungefär.

for x in range(1000):
  for y in range(1000):
    r = min(255, red[x][y]/divide)
    g = min(255, green[x][y]/divide)
    b = min(255, blue[x][y]/divide)
    rgb[x][y] = [r,g,b]

img = Image.fromarray(rgb)
display(img)

Testa att ändra variabeln divide ovan. Vad händer med bilden? 

<details>
<summary markdown="span">
Svar
</summary>
<p>
Hur ljus bilden är ändras. Om `divide` är liten blir bilden väldigt ljus, om `divide` är stor blir den väldigt mörk. `divide` styr i princip hur stor skillnad de är på de olika färgerna i bilden. 

För att få en sanningsenlig bild ska `divide` väljas till 14, då blir dock bilden ganska mörk. För 8 är det lättare att se de olika nyanserna i bilden.
</p>
</details>

Eftersom vi ska räkna sjöar i bilden vill vi ha ett bra sätt att veta vad som är vatten. Vi kommer använda oss av NDVI som du bör ha stött på i tidigare uppgifter. NDVI kollar på hur det nära infraröda bandet skiljer sig från det röda bandet.

In [0]:
ndvi = (nir-red)/(nir+red) #Skapar en ny variabel 'ndvi' med samma format som de andra banden.

plt.figure(figsize=(10,10))
plt.pcolormesh(ndvi, cmap='PiYG')
plt.ylim(ndvi.shape[0], 0)
plt.clim(-1.0, 1.0)
plt.colorbar(label='NDVI')
plt.show()

NDVI blir ett tal mellan -1 och 1. I bilden ser vi att vatten blir rosa, det vill säga NDVI är litet  för vatten. En regel vi kan ha är att om NDVI är mellan -1 och -0.1 så är det vatten i pixeln.

Varje pixel i bilden motsvarar ett område som är $50*50$ meter. Som man kan se av skalan så är det $1000*1000$ pixlar i bilden.

Hur brett och högt är området på bilden?

<details>
<summary markdown="span">
Lösning
</summary>
<p>
Varje pixel är $50m$ bred, totalt är området $50*1000 = 50000 m = 50 km$ brett. Det samma gäller höjden på bilden.
</p>
</details>

Hur stor är arean av hela området?

<details>
<summary markdown="span">
Lösning
</summary>
<p>
$50 km * 50 km = 2500 km^2$
</p>
</details>

Ungfär hur många sjöar/vattendrag tror du att det är?

<details>
<summary markdown="span">
Svar
</summary>
<p>
Detta kan vi förstås inte skriva ett svar på här, det är ju det vi ska räkna ut i resten av uppgiften! Dock kanske du kan konstatera att det verkar vara väldigt många, fler än du skulle kunna/vilja räkna för hands.
</p>
</details>

För att slippa tänka på NDVI i fortsättningen, vill vi göra en tabell som är `True` för pixlar det är vatten, och `False` annars. Fyll i koden nedan så att tabellen `water` beskriver just detta. 

In [0]:
water = np.zeros([1000, 1000], dtype=np.bool)
for x in range(1000):
  for y in range(1000):
    if ...: # Skriv ett vilkor så att vi vet om det är vatten här eller inte.
      water[x][y] = True
    else:
      water[x][y] = False

<details>
<summary markdown="span">
Tips
</summary>
<p>
Givet värdet på NDVI, hur vet vi om det är vatten då?
</p>
</details>

<details>
<summary markdown="span">
Svar
</summary>
<p>

```python
water = np.zeros([1000, 1000], dtype=np.bool)
for x in range(1000):
  for y in range(1000):
    if ndvi[x][y] < -0.1: # Skriv ett vilkor så att vi vet om det är vatten här eller inte.
      water[x][y] = True
    else:
      water[x][y] = False
```
</p>
</details>

Vi kan nu göra om tabellen till en svartvit bild.

In [0]:
waterpic = np.zeros([1000, 1000], dtype=np.uint8)

for x in range(1000):
  for y in range(1000):
    if water[x,y]:
      waterpic[x,y]=255


img = Image.fromarray(waterpic)
display(img)

Nu ska vi släppa vattendragen en liten stund och istället gå in lite på djupet i hur vi ska lösa det här problemet med programmering.

### Att hitta komponenter med hjälp av DFS

I uppgifterna ovan har vi kommit fram till en tabell som i varje pixel säger oss om det är vatten där eller inte. Många pixlar med vatten kommer att "sitta ihop" och tillsammans bilda en sjö. Vi behöver nu på något sätt klumpa ihop pixlarna så att vi vet vilka pixlar som sitter ihop med varandra. Vi kan kalla pixlar som sitter ihop för en komponent. 

Vi kan tänka att vi ska börja i en pixel med vatten, och sen gå ut från den så att vi hittar alla pixlar den sitter ihop med. I varje steg så har vi en nuvarande pixel och tittar på alla dess närliggande pixlar. De som vi inte varit på innan och som är vatten lägger vi till i en lista som håller reda på vilka pixlar vi har kvar att besöka. Sen påbörjar vi ett nytt steg där vi tar bort den sista pixeln i listan och väljer till vår nuvarande punkt. Sen fortsätter vi på det sättet 



Vi börjar med att göra en stor tabell som för varje pixel håller reda på vilken sjö den tillhör. I början är alla värden 0 i tabellen vilket betyder att pixeln inte hamnat i en sjö än.

In [0]:
lakenumbers = np.zeros([1000, 1000], dtype=np.uint16)

Skriv en hjälp-funktion som kollar om en punkt ligger i bilden. Tänk på att bilden är $1000*1000$ pixlar stor.

In [0]:
#Blir True om punkten (x,y) ligger i bilden, annars False
def in_picture(x,y):
  # Skriv din kod här:

<details>
<summary markdown="span">
Tips
</summary>
<p>
Både x och y ska vara ett värde mellan 0 och 999.
</p>
</details>
<details>
<summary markdown="span">
Lösning
</summary>
<p>

```python
def in_picture(x,y):
  return 0 <= x < 1000 and 0 <= y < 1000
```
</p>
</details>

Nu ska du snart skriva en funktion som utgår från en punkt och hittar alla punkter med vatten som sitter ihop med den och markerar dessa med talet lakenumber. 

Psuedokod finns inskrivet, som säger vad som ska göras i varje steg. 

Det kan vara några begrepp du inte sett innan eller som du behöver friska upp minnet för. 

#### Begrepp som kan vara användbara

Tuplar är ett enkelt sätt att gruppera saker i python.  

*   De fungerar ungefär som listor men kan inte ändras.
*   Skriver man `a = (5,2,"hejsan")` så blir `a` en tupel med talen 5 och 2 samt strängen `"hejsan"`.
*   Man kan få ut respektive element på samma sätt som med en lista: `print(a[1])` skriver ut `2`.
*   Man kan dock inte ändra i tupeln: `a[2] = "tjena"` ger ett fel.
*   Man kan 'packa upp' tupeln genom att skriva: `x,y,z = a`, så blir `x`, `y` och `z` vars ett av elementen som ligger i `a`.

Testa att packa upp tuplen nedan i rätt ordning.

In [0]:
a = ("dig!", "på", "Hej")

#Skriv kod här för att packa upp tuplen i rätt ordning!

print(x,y,z) # Utskriften ska bli: Hej på dig!


Pop är en funktion som tar ut det sista elementet i en lista.

*   `last = list.pop()` Tar bort det sista elementet ur `list` och lägger i variabeln `last`.

Vad skriver följande kodcell ut? Förstår du varför?

In [0]:
a = ["dig!", "på", "Hej"]
print(a.pop(), a.pop(), a.pop())


<details>
<summary markdown="span">
Lösning
</summary>
<p>
Då pop plockar ut det sista elementet ur listan kommer det vara ett nytt element som ligger sist i listan 'a' varje gång vi anropar pop.  
</p>
</details>







En while-loop körs så länge ett visst villkor är uppfyllt.
Till exempel:
```
i = 10
while i > 5:
  print(i)
  i = i-1
```

Skriver ut talen `10, 9, 8, 7, 6` på vars en rad.

Kan du skriva ett villkor till while-loopen nedan så att den kör tills listan är tom?

In [0]:
a = ["!","g","i","d"," ","å","p"," ","j","e","H"]

while ... : #Byt ut ... mot ett villkor.
  print(a.pop())

<details>
<summary markdown="span">
Tips
</summary>
<p>
Du kan exempelvis kolla om längden av listan 'a' är större än 0. 
</p>
</details>
<details>
<summary markdown="span">
Lösning
</summary>
<p>

```python

while len(a) > 0 : 
  print(a.pop())

```

</p>
</details>

#### Skriv funktion för att hitta komponenter med DFS

Fyll i kod som det finns psuedokod för här nedan.

In [0]:
def find_component(startx, starty, lakenumber):
  #Skapa en lista points och stoppa in tupeln (startx,starty) i den
  #Sätt att lakenumbers[startx][starty] är lakenumber
  #så länge points inte är tom gör följande:
    #Plocka bort den sista punkten i points och lägg i variablerna x och y
    #Skapa en lista med alla närliggande punkter som gränsar till (x, y) (upp,ner,vänster,höger)
    #Skapa en loop som går igenom listan av grannar:
      #kalla den nya punkten för (nx, ny)
      #om (nx,ny) ligger i bilden, är vatten, och det står 0 i lakenumbers[nx][ny]:
        #Sätt att lakenumbers[nx][ny] är lakenumber
        #Lägg in tupeln (nx,ny) längst bak i listan points.



<details>
<summary markdown="span">
Om det är för svårt kan du istället börja med följande kodskelett
</summary>
<p>

```python
def find_component(startx, starty, lakenumber):
  points = [...]
  lakenumbers[startx][starty] = ...
  while ...:
    x,y = ... 
    neighbours = [(... , ...),(... , ...),(... , ...),(... , ...)]
    for neighbour in ...:
      nx = ...
      ny = ...
      if ... and ... and ... :
        lakenumbers[nx][ny] = ...
        points.append(...)
```
</p>
</details>
<details>
<summary markdown="span">
Tips
</summary>
<p>
Begreppen som vi tidigare kollade på kan komma till stor användning. 
Vi kan använda tuplar för att hålla koll på x och y kordinater, och packa upp dem när det behövs.
Vi såg att pop kan användas för att plocka ut det sista elementet. 
En while-loop kan användas för att göra något flera gånger så länge ett villkor är uppfyllt.

</p>
</details>
<details>
<summary markdown="span">
Lösning
</summary>
<p>

```python
def find_component(startx, starty, lakenumber):
  points = [(startx,starty)]
  lakenumbers[startx][starty] = lakenumber
  while len(points) > 0:
    x,y = points.pop()
    neighbours = [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]
    for neighbour in neighbours:
      nx = neighbour[0]
      ny = neighbour[1]
      if in_picture(nx,ny) and water[nx][ny] and lakenumbers[nx][ny] == 0:
        lakenumbers[nx][ny] = lakenumber
        points.append((nx,ny))
```
</p>
</details>

När koden är färdig kan vi testa att funktionen fungerar som den ska. Följande kod borde visa hur vi hittar en sjö uppe till vänster. Tar det väldigt lång tid utan att hända något kan din funktion ovanför vara felaktig.

In [0]:
lakenumbers = np.zeros([1000, 1000], dtype=np.uint8)
find_component(0,30,255)

img = Image.fromarray(lakenumbers)
display(img)

Det hade varit smidigt om vi visste hur stor sjön vi hittar är när vi anropar `find_component`. Längre fram vill vi nämligen utesluta de komponenter som är för små för att räknas som sjöar.

Kopiera in din tidigare kod för `find_component` och ändra så att den returnerar antalet pixlar i komponenten som hittades.

<details>
<summary markdown="span">
Tips
</summary>
<p>

Ha en variabel som räknar upp 1 varje gång en punkt/pixel läggs in i listan points. Returnera variabeln i slutet av funktionen.
</p>
</details>



In [0]:
# Kopiera in din tidigare kod här, och ändra den.


<details>
<summary markdown="span">
Lösning
</summary>
<p>

```python
def find_component(startx, starty, lakenumber):
  points = [(startx,starty)]
  component_size = 1
  lakenumbers[startx][starty] = lakenumber
  while len(points) > 0:
    x,y = points.pop()
    steps = [(1,0),(-1,0),(0,1),(0,-1)]
    for step in steps:
      nx = x+step[0]
      ny = y+step[1]
      if in_picture(nx,ny) and water[nx][ny] and lakenumbers[nx][ny] == 0:
        lakenumbers[nx][ny] = lakenumber
        points.append((nx,ny))
        component_size += 1
  return component_size
```
</p>
</details>

Vad är en sjö och vad är en pöl? I sverige har vi satt gränsen vid 1 hektar. Det vill säga om vattensamlingen är större än ett område som är $100 * 100$ meter så räknas det som en sjö. 
Om du minns så representerade pixlarna i bilden ett område på $50 * 50$ meter. Hur många pixlar krävs för att vattensamlingen ska räknas som en sjö? Fyll i i koden nedan och kör för att se hur många sjöar det är på bilden.

In [0]:

lakenumbers = np.zeros([1000, 1000], dtype=np.uint16)

min_number_pixels = ??? # Fyll i minsta antal pixlar för en sjö

lakenumber = 1
lakesfound = 0
for x in range(1000):
  for y in range(1000):
    if water[x][y] and lakenumbers[x][y] == 0:
      if find_component(x,y,lakenumber)>=min_number_pixels:
        lakesfound+=1
      lakenumber += 1
print ("Antal sjöar hittade:")
print (lakesfound)

Hur många sjöar finns det totalt i bilden? 

<details>
<summary markdown="span">
Svar
</summary>
<p>
Det ska finnas 674 sjöar totalt.
</p>
</details>




Testa att ändra gränsen för hur stor en sjö måste vara. Hur många sjöar blir det om det räcker med en pixel för att räknas som en sjö?

Enligt [SMHI](https://www.smhi.se/kunskapsbanken/hydrologi/sveriges-sjoar-1.4221) finns det totalt nästan 100000 sjöar i Sverige. Jämför detta med antalet sjöar vi hittade i vår bild. Verkar det rimligt? Du kan scrolla upp och titta på kartan längst upp för att se hur stort område satellitbilden täckte.

<details>
<summary markdown="span">
Svar
</summary>
<p>
De 674 sjöarna vi hittade motsvarar knappt 1% av alla sveriges sjöar. Om vi scrollar upp och tittar på kartan över Sverige verkar detta rimligt, satellitbilden täcker ungefär så stor del av Sveriges yta.
</p>
</details>